<a href="https://colab.research.google.com/github/TrungKhoaLe/housing-price-prediction/blob/main/notebooks/housing_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Business understanding

The goal of this project is to predict the median housing price of a district and the predicted outputs will be fed to a downstream machine learning system that yields investment decisions.

Candidate algorithms to explore:

- Linear Regression
- Decision Trees
- Random Forests

Metrics to evaluate the models:

- Root Mean Squared Error (RMSE)

# Get the data

In [1]:
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request

def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing/housing.csv"))

# run the function
housing = load_housing_data()

## Glance the data info

In [2]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


## Create test set

The following are the ways that can be used to split a dataset:

- Splitting the dataset using index slicing,
- computing the hash value for each instance and put that instance in the test set if the hash is lower than or equal to 20% of the maximum hash value,
- using the Scikit-Learn's `train_test_split()`,
- using the *stratified* sampling method to avoid a sampling bias.

In [3]:
import numpy as np

housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6.0, np.inf],
                               labels=[1, 2, 3, 4, 5])

In [4]:
import plotly.express as px

fig = px.bar(housing, x="income_cat")
fig.update_traces(marker_color="red",
                  opacity=1)
fig.show()

In [5]:
from sklearn.model_selection import train_test_split

strat_train_set, strat_test_set = train_test_split(housing, test_size=0.2,
                                                   stratify=housing["income_cat"], random_state=42)
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# EDA

In [6]:
# NOTE: if the dataset is huge, we might want to sample
# an exploration dataset for quick iteration purposes
housing_eda = strat_train_set.copy()

## Visualizing geographical features

In [7]:
fig = px.scatter(housing_eda, x="longitude", y="latitude",
                 size="population", color="median_house_value")
fig.show()

## Looking for correlations


In [8]:
import plotly.figure_factory as ff

attributes = ["median_house_value", "median_income", "total_rooms",
"housing_median_age"]
fig = ff.create_scatterplotmatrix(housing_eda[attributes], diag='histogram',
                                  colormap=['rgb(100, 150, 255)', '#F0963C', 'rgb(51, 255, 153)'],
                                  colormap_type='seq', height=800, width=800, marker_opacity=0.1)
fig.show()

A strong correlation has occured between the `median_income` feature and the `target`, meaning that the `median_income` feature might have strong predictive power.

In [9]:
fig = px.scatter(housing_eda, x="median_income", y="median_house_value")
fig.update_traces(marker_opacity=0.1)
fig.show()

# Prepare the data

In [10]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

## Feature scaling

**Input features**

When a feature's distribution has a heavy tail, both min-max scaling and standardization will squash most values into a small range. Machine learning algorithms do not like this at all.

So before performing feature scaling, we should first transform it to shrink the heavy tail, and if possible to make the distribution roughly symmetrical.

We apply the following ways:

- Replacing the feature with is square root (or raising the feature to a power between 0 and 1) if the feature is positive,

- replacing the feature with its logarithm if the feature has a really long and heavy tail, such as a *a power law distribution*,

- `bucketizing` the feature. This means chopping its distribution into roughly equal-sized buckets, and replacing each feature value with the index of the bucket it belongs to. Bucketizing with equal-sized buckets results in a feature with an almost uniform distribution, so there's no need for further scaling, or we can just divide by the number of buckets to force the values into the 0-1 range,

- when a feature has a multimodal distribution (e.g. with two or more clear peaks, called `modes`), it can be helpful to `bucketize` it,

- multimodal distribution features can be transformed by adding a feature for each of the `modes`, representing the similarity between the feature and that particular mode.

**Target values**

When the target distribution has a heavy tail, we may choose to replace the target with its logarithm. But, if you do, the regression model will now predict the *log* of the target value, not the target value itself. We will need to compute the exponential ofthe model's prediction if we want the original-scale predicted value. To perform all of this, we can use the `TransformedTargetRegressor` class of Scikit-Learn.

## Custom transformers

For transformations that do not require any training, we can use the `FunctionTransformer` class of Scikit-Learn.

If we want a trainable transformer, we can make use of the `BaseEstimator` and `TransformerMixin` classes to define a custom transformer class.

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import rbf_kernel


class ClusterSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters=10, gamma=1.0, random_state=None):
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.random_state = 42
    
    def fit(self, X, y=None, sample_weight=None):
        self.kmeans_ = KMeans(self.n_clusters, random_state=self.random_state)
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self
    def transform(self, X):
        return rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)

    def get_feature_names_out(self, names=None):
        return [f"Cluster {i} similarity" for i in range(self.n_clusters)]

## Transformation pipeline

Scikit-Learn provides the `Pipeline` class that helps with sequences of transformations.

The `Pipeline` constructor takes a list of name/estimator pairs defining a sequence of steps. The name can be anything we like, as long as they are unique and does not contain double underscores (__). The estimators must all be transformers, except for the last one, which can be anything: a transformer, a predictor, or any other type of estimator.

If we do not want to name the transformers, we can use the `make_pipeline` function.

In [12]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector

def column_ratio(X):
    return X[:, [0]] / X[: ,[1]]

def ratio_name(function_transformer, feature_names_in):
    return ["ratio"]

def ratio_pipeline():
    return make_pipeline(
        SimpleImputer(strategy="median"),
        FunctionTransformer(column_ratio, feature_names_out=ratio_name),
        StandardScaler()
    )


log_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log, feature_names_out="one-to-one"),
    StandardScaler()
)

cluster_simil = ClusterSimilarity(n_clusters=10, gamma=1.0, random_state=42)
defaul_num_pipeline = make_pipeline(SimpleImputer(strategy="median"),
                                    StandardScaler())
cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore")
)

preprocessing = ColumnTransformer([
    ("bedrooms", ratio_pipeline(), ["total_bedrooms", "total_rooms"]),
    ("rooms_per_house", ratio_pipeline(), ["total_rooms", "households"]),
    ("people_per_house", ratio_pipeline(), ["population", "households"]),
    ("log", log_pipeline, ["total_bedrooms", "total_rooms", "population",
                           "households", "median_income"]),
    ("geo", cluster_simil, ["latitude", "longitude"]),
    ("cat", cat_pipeline, make_column_selector(dtype_include=object)),
], 
remainder=defaul_num_pipeline)

# Modeling

In [13]:
import warnings
warnings.filterwarnings("ignore")

## Create a baseline

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = make_pipeline(preprocessing, LinearRegression())
lin_reg.fit(housing, housing_labels)

# evaluate the model on the traing set
lin_rmse = mean_squared_error(housing_labels,
                              lin_reg.predict(housing),
                              squared=False)

lin_rmse

68687.89176590038

|                    	| Min      	    | Max      	    | RMSE        	|
|--------------------	|-----------	|-----------	|-------------	|
| median_house_value 	| \$120,000 	| \$265,000 	| \$68,687.892 	|

From the table, it is clear that the error rate is significantly large.

In [17]:
import joblib

joblib.dump(lin_reg, "model-linear.joblib")

['model-linear.joblib']

## Train and evaluate models with cross-validation

### Decision trees

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

tree_reg = make_pipeline(preprocessing, DecisionTreeRegressor(random_state=42))

tree_rmses = -cross_val_score(tree_reg, 
                              housing,
                              housing_labels,
                              scoring="neg_root_mean_squared_error",
                              cv=10)

In [16]:
tree_rmses.mean()

66868.02728758389

In [18]:
import joblib

joblib.dump(tree_reg, "model-decision-tree.joblib")

['model-decision-tree.joblib']

### Random forests

In [19]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = make_pipeline(preprocessing, RandomForestRegressor(random_state=42))
forest_rmses = -cross_val_score(forest_reg,
                                housing,
                                housing_labels,
                                scoring="neg_root_mean_squared_error",
                                cv=10)

In [20]:
forest_rmses.mean()

47019.56128114021

# Fine-tune the model

## Randomized search

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint


full_pipeline = Pipeline(
    [("preprocessing", preprocessing),
    ("random_forest", RandomForestRegressor(random_state=42))
])

param_distribs = {'preprocessing__geo__n_clusters': randint(low=3, high=50),
                  'random_forest__max_features': randint(low=2, high=20)}

rnd_search = RandomizedSearchCV(
    full_pipeline,
    param_distributions=param_distribs,
    n_iter=10,
    cv=10,
    scoring='neg_root_mean_squared_error',
    random_state=42)

rnd_search.fit(housing, housing_labels)          

RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                                                           SimpleImputer(strategy='median')),
                                                                                          ('standardscaler',
                                                                                           StandardScaler())]),
                                                                transformers=[('bedrooms',
                                                                               Pipeline(steps=[('simpleimputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('functiontransformer',
                                                                                                FunctionTransformer(feature_names...
                                             ('random_forest',
                                              RandomForestRegressor(random_state=42))]),
                   param_distributions={'preprocessing__geo__n_clusters': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f0920fee100>,
                                        'random_forest__max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f0920fee1c0>},
                   random_state=42, scoring='neg_root_mean_squared_error')

In [22]:
cv_res = pd.DataFrame(rnd_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)
cv_res = cv_res[["param_preprocessing__geo__n_clusters",
                 "param_random_forest__max_features", "split0_test_score",
                 "split1_test_score", "split2_test_score", "split3_test_score",
                 "split4_test_score", "split5_test_score", "split6_test_score",
                 "split7_test_score", "split8_test_score", "split9_test_score", 
                 "mean_test_score"]]

score_cols = ["split0", "split1", "split2", "split3", "split4", "split5",
              "split6", "split7", "split8", "split9", "mean_test_rmse"]
cv_res.columns = ["n_clusters", "max_features"] + score_cols
cv_res[score_cols] = -cv_res[score_cols].round().astype(np.int64)
cv_res.head()

,n_clusters,max_features,split0,split1,split2,split3,split4,split5,split6,split7,split8,split9,mean_test_rmse
1,45,9,41110,41176,38949,40385,40314,42747,40867,42899,41494,42112,41205
8,32,7,41351,41379,39178,40546,40652,42824,41190,43526,42042,42394,41508
5,42,4,41933,41745,38914,40689,41040,43113,41238,43747,42142,42965,41753
0,41,16,41812,42390,40321,41027,40986,43167,41797,44202,42455,42379,42054
2,23,8,42266,42108,40254,41652,41349,43201,42025,44044,42325,42373,42160


## Error analysis

In [23]:
final_model = rnd_search.best_estimator_
feature_importances_ = final_model["random_forest"].feature_importances_

In [24]:
sorted(zip(feature_importances_, 
    final_model["preprocessing"].get_feature_names_out()),
    reverse=True)

[(0.18694559869103852, 'log__median_income'),
 (0.0748194905715524, 'cat__ocean_proximity_INLAND'),
 (0.06926417748515576, 'bedrooms__ratio'),
 (0.05446998753775219, 'rooms_per_house__ratio'),
 (0.05262301809680712, 'people_per_house__ratio'),
 (0.03819415873915732, 'geo__Cluster 0 similarity'),
 (0.02879263999929514, 'geo__Cluster 28 similarity'),
 (0.023530192521380392, 'geo__Cluster 24 similarity'),
 (0.020544786346378206, 'geo__Cluster 27 similarity'),
 (0.019873052631077512, 'geo__Cluster 43 similarity'),
 (0.018597511022930273, 'geo__Cluster 34 similarity'),
 (0.017409085415656868, 'geo__Cluster 37 similarity'),
 (0.015546519677632162, 'geo__Cluster 20 similarity'),
 (0.014230331127504292, 'geo__Cluster 17 similarity'),
 (0.0141032216204026, 'geo__Cluster 39 similarity'),
 (0.014065768027447325, 'geo__Cluster 9 similarity'),
 (0.01354220782825315, 'geo__Cluster 4 similarity'),
 (0.01348963625822907, 'geo__Cluster 3 similarity'),
 (0.01338319626383868, 'geo__Cluster 38 similarity'

## Evaluate the model on the test set

In [25]:
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

final_predictions = final_model.predict(X_test)
final_mse = mean_squared_error(y_test, final_predictions, squared=False)
final_mse

41424.40026462184

In [26]:
import joblib

joblib.dump(final_model, "model-random-forest.joblib")

['model-random-forest.joblib']